In [9]:
import torch
torch.__version__

'1.3.1'

In [18]:
def subidx(n:int, remove:int) -> list:
    out = list(range(n))
    out.remove(remove)
    return out

def submatrix(A:torch.tensor, j:int) -> torch.tensor:
    n = A.shape[0]
    s = subidx(n, j)
    o = A[s, :]
    o = o[:, s]
    return o

def get_eigenvector_val_old(hermitian_matrix, i, j):
    lam, v = torch.eig(hermitian_matrix, eigenvectors=True)
    old_eigenvector_ij = torch.abs(v[j,i]**2)
    
    print(f'Old eigenvector ij: {old_eigenvector_ij}')

def get_eigenvector_val(hermitian_matrix, i, j):
    # New way: torch
    lam, v = torch.eig(hermitian_matrix, eigenvectors=False)

    n = len(lam)
    M = submatrix(hermitian_matrix, j)
    lam_submatrix, _ = torch.eig(M)

    # Remove 0's
    lam = lam[:, 0]
    lam_submatrix = lam_submatrix[:, 0]
    
    # Left side of equation 2
    left = torch.prod(torch.tensor([lam[i] - lam[k] for k in range(n) if k!=i]))
    # Right side of equation 2
    right = torch.prod(torch.tensor([lam[i] - lam_submatrix[k] for k in range(n-1)]))
    # Right divided by left
    eigenvector_ij = right / left
    
    print(f'New Eigenvector ij: {eigenvector_ij}')

In [19]:
# Random square matrix
rand_square_matrix = torch.rand(50, 50)

# Hermitian matrix
hermitian_matrix = rand_square_matrix * rand_square_matrix.T

# Old 
get_eigenvector_val_old(hermitian_matrix, i=2, j=2)
print('-'*50)

# New
get_eigenvector_val(hermitian_matrix, i=2, j=2)

Old eigenvector ij: 0.002864641835913062
--------------------------------------------------
New Eigenvector ij: 0.002861912827938795
